<a href="https://colab.research.google.com/github/peytonrunyan/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/LS_DS_142_Sampling_Confidence_Intervals_and_Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 142
## Sampling, Confidence Intervals, and Hypothesis Testing

## Prepare - examine other available hypothesis tests

If you had to pick a single hypothesis test in your toolbox, t-test would probably be the best choice - but the good news is you don't have to pick just one! Here's some of the others to be aware of:

In [0]:
# Distribution tests:
# We often assume that something is normal, but it can be important to *check*

# For example, later on with predictive modeling, a typical assumption is that
# residuals (prediction errors) are normal - checking is a good diagnostic

from scipy.stats import normaltest
# Poisson models arrival times and is related to the binomial (coinflip)
sample = np.random.poisson(5, 1000)
print(normaltest(sample))  # Pretty clearly not normal

NormaltestResult(statistic=32.52084485715453, pvalue=8.673377835963947e-08)


In [0]:
# Kruskal-Wallis H-test - compare the median rank between 2+ groups
# Can be applied to ranking decisions/outcomes/recommendations
# The underlying math comes from chi-square distribution, and is best for n>5
from scipy.stats import kruskal

x1 = [1, 3, 5, 7, 9]
y1 = [2, 4, 6, 8, 10]
print(kruskal(x1, y1))  # x1 is a little better, but not "significantly" so

x2 = [1, 1, 1]
y2 = [2, 2, 2]
z = [2, 2]  # Hey, a third group, and of different size!
print(kruskal(x2, y2, z))  # x clearly dominates

KruskalResult(statistic=0.2727272727272734, pvalue=0.6015081344405895)
KruskalResult(statistic=7.0, pvalue=0.0301973834223185)


And there's many more! `scipy.stats` is fairly comprehensive, though there are even more available if you delve into the extended world of statistics packages. As tests get increasingly obscure and specialized, the importance of knowing them by heart becomes small - but being able to look them up and figure them out when they *are* relevant is still important.

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):

1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.

In [0]:
# Getting started with drug data
# http://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Drugs.com%29

!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip

--2018-12-04 21:42:00--  http://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/zip]
Saving to: ‘drugsCom_raw.zip.2’

drugsCom_raw.zip.2  100%[===================>]  41.00M  19.0MB/s    in 2.2s    

2018-12-04 21:42:03 (19.0 MB/s) - ‘drugsCom_raw.zip.2’ saved [42989872/42989872]



In [0]:
!unzip drugsCom_raw.zip

Archive:  drugsCom_raw.zip
replace drugsComTest_raw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace drugsComTrain_raw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
!head drugsComTrain_raw.tsv

In [0]:
import pandas as pd
df = pd.read_table('drugsComTrain_raw.tsv')
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [0]:
df.shape

(161297, 7)

In [0]:
import scipy.stats as stats
#stats.norm.interval(0.68, loc=mu, scale=sigma/sqrt(N))

In [0]:
drugs = list(df['drugName'])

In [0]:
import numpy as np

In [0]:
for item in drugs:
  mean = np.mean(df[df['drugName'] == item ]['rating'])
  count = len(df[df['drugName'] == item ]['rating'])
  
# drux = 'Valsartan'
# mean = np.mean(df[df['drugName'] == drux ]['rating'])
# count = len(df[df['drugName'] == drux ]['rating'])

# print(mean)
# print(count)

In [0]:
df.isna().sum()

Unnamed: 0       0
drugName         0
condition      899
review           0
rating           0
date             0
usefulCount      0
dtype: int64

In [0]:
df.groupby(['drugName']).mean[1]

In [0]:
drug_df.head()

In [0]:
drugs_n = df.groupby(['drugName'])['condition'].count()
len(drugs_n)

In [0]:
drugs_n[0]

In [0]:
rating_mean = df.groupby(['drugName'])['rating'].mean()

In [0]:
#CI = sample mean +- interval
# interval = t_value * (sample_sd/sqrt(sampe_size)
# alpha = desired cutoff for p 
t_value = stats.t.ppf(1)

10.0

In [0]:
rating_stddev = df.groupby('drugName')['rating'].std()

In [0]:
#rating_mean
#drugs_n
#rating_stddev

In [0]:
alpha = .05
conf_int= []
for i in range(len(drugs_n)-1):
  t = stats.t.ppf(1-(alpha/2), (drugs_n[i])-1)
  x_bar = rating_mean[i]
  s = rating_stddev[i]
  n = drugs_n[i]
  sample_error = (t*s)/np.sqrt(n)
  upper = x_bar+sample_error
  lower = x_bar-sample_error
  conf_int.append((lower,upper))
  

In [0]:
len(conf_int)
#np.std(drug_df['Valsartan'])


In [0]:
len(df)

In [0]:
conf_int


[(nan, nan),
 (nan, nan),
 (7.517805249284367, 8.905271673792555),
 (nan, nan),
 (5.509542281611813, 8.806247192072398),
 (6.224121370392706, 6.856596118396532),
 (8.0, 8.0),
 (0.5894958131104087, 14.910504186889591),
 (7.600700678337309, 10.621521543884914),
 (2.6973472700887253, 11.302652729911275),
 (0.5640190810107528, 14.435980918989248),
 (5.106685314323055, 6.349010888208591),
 (nan, nan),
 (-51.67792131394443, 62.67792131394443),
 (8.515915184166193, 9.282249953448485),
 (6.447677567494251, 8.396766876950194),
 (nan, nan),
 (10.0, 10.0),
 (nan, nan),
 (8.152721283895652, 8.701824170649804),
 (3.146897631783953, 15.853102368216046),
 (6.887722108246085, 9.178944558420582),
 (nan, nan),
 (6.971292608107678, 8.316063713731403),
 (1.162245450147875, 15.504421216518793),
 (8.043561120599888, 9.756438879400113),
 (8.489710725562476, 9.308738886840624),
 (8.366841475956974, 9.868452641690084),
 (8.065782423362908, 10.934217576637092),
 (6.811057696807939, 9.903228017477776),
 (8.21893